## Second Order Taskonomy

In [60]:
import pandas as pd
import numpy as np
import pickle
bucket_type = 'top5'

metrics = pd.DataFrame(columns=['source_type', 'target_type', 'stats'])

type_list_path = f'/l/users/shikhar.srivastava/data/pannuke/{bucket_type}/selected_types.csv'
type_list = pd.read_csv(type_list_path)['0']

for source_type in type_list:
    for target_type in type_list:
        logs_path = f'/l/users/shikhar.srivastava/workspace/hover_net/logs/test/second_order/{bucket_type}/ckpts/{source_type}-{target_type}/per_image_stat.pkl'
        # Read pickle file
        with open(logs_path, 'rb') as f:
            stats = pickle.load(f)
        
        metrics = metrics.append({'source_type': source_type, 'target_type': target_type, 'stats': stats}, ignore_index=True)


In [65]:
stats[0]

{'np_acc': 0.9943486020226056,
 'np_dic': 0.8931082981653087,
 'tp_dic_0': 0.9959467727131777,
 'tp_dic_1': 0.0,
 'tp_dic_2': 0.0,
 'tp_dic_3': 0.8575268817146672,
 'tp_dic_4': 0.0,
 'tp_dic_5': 0.0,
 'hv_mse': 0.009358107086308722}

In [92]:

def compute_winrate_ij(source_i, source_j, target_type, metrics):
    metrics_source_i = metrics[(metrics['source_type'] == source_i) & (metrics['target_type'] == target_type)]['stats'].iloc[0]
    metrics_source_j = metrics[(metrics['source_type'] == source_j) & (metrics['target_type'] == target_type)]['stats'].iloc[0]
    n_images = len(metrics_source_i)
    wins = 0
    losses = 0
    for img_idx in range(n_images):
        acc_i = metrics_source_i[img_idx]['np_acc']
        acc_j = metrics_source_j[img_idx]['np_acc']
        dice_i = metrics_source_i[img_idx]['np_dic']
        dice_j = metrics_source_j[img_idx]['np_dic']
        if (acc_i > acc_j) & (dice_i > dice_j):
            wins += 1
        else:
            losses += 1
    winrate = wins/n_images
    losses = losses/n_images
    return winrate #, losses



In [115]:
def tournament_matrix(type_list, metrics):
    W = np.zeros(shape = (len(type_list), len(type_list) - 1, len(type_list) - 1))
    labels = []
    for t, target_type in enumerate(type_list):
        sources = set(type_list)
        sources.remove(target_type)

        print('target_type: ', target_type)
        n_sources = len(sources) # All but the target, can be source types
        W_t = np.zeros((n_sources, n_sources))
        labels_t = list(sources)

        for i, source_i in enumerate(sources):
            for j, source_j in enumerate(sources):
                #print(source_i, source_j)
                W_t[i, j] = compute_winrate_ij(source_i, source_j, target_type, metrics)
                #acc_pivot.loc[source_i, target_type] - acc_pivot.loc[source_j, target_type]
        W_t = np.clip(W_t, 1e-3, 1 - 1e-3)
        W_t_ = W_t.copy()/W_t.copy().T
        W[t, :, :] = W_t_
        labels.append(labels_t)
    return W, labels

In [116]:
W, labels = tournament_matrix(type_list, metrics)

target_type:  Breast
target_type:  Colon
target_type:  Adrenal_gland
target_type:  Esophagus
target_type:  Bile-duct


In [128]:
W

array([[[1.        , 0.87931034, 0.99404762, 0.92352941],
        [1.1372549 , 1.        , 1.01818182, 1.20945946],
        [1.00598802, 0.98214286, 1.        , 1.14379085],
        [1.08280255, 0.82681564, 0.87428571, 1.        ]],

       [[1.        , 0.87978142, 0.70813397, 1.02339181],
        [1.13664596, 1.        , 0.81443299, 1.19745223],
        [1.41216216, 1.2278481 , 1.        , 1.45070423],
        [0.97714286, 0.83510638, 0.68932039, 1.        ]],

       [[1.        , 0.645     , 0.84745763, 0.72251309],
        [1.5503876 , 1.        , 1.49618321, 1.34265734],
        [1.18      , 0.66836735, 1.        , 0.88439306],
        [1.38405797, 0.74479167, 1.13071895, 1.        ]],

       [[1.        , 1.0974026 , 1.01863354, 1.12337662],
        [0.9112426 , 1.        , 0.81420765, 1.06493506],
        [0.98170732, 1.22818792, 1.        , 1.515625  ],
        [0.89017341, 0.93902439, 0.65979381, 1.        ]],

       [[1.        , 1.20253165, 1.40277778, 1.27152318],
      

In [127]:
np.linalg.eig(W[2])

(array([ 4.00320540e+00+0.j        , -1.72544722e-03+0.11326852j,
        -1.72544722e-03-0.11326852j,  2.45491380e-04+0.j        ]),
 array([[ 0.38207979+0.j        ,  0.16157098+0.42589459j,
          0.16157098-0.42589459j,  0.0543442 +0.j        ],
        [ 0.64022543+0.j        , -0.74317369+0.j        ,
         -0.74317369-0.j        ,  0.21433985+0.j        ],
        [ 0.44000302+0.j        ,  0.08143593-0.18209174j,
          0.08143593+0.18209174j, -0.75428852+0.j        ],
        [ 0.50052351+0.j        ,  0.27714835-0.35156943j,
          0.27714835+0.35156943j,  0.61818603+0.j        ]]))